In [1]:
import torch
print(torch.cuda.is_available())
import warnings
warnings.filterwarnings('ignore')

True


In [2]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

#AutoModelForQuestionAnswering: A class to load pre-trained models for question-answering tasks.
#AutoTokenizer: A class to load pre-trained tokenizers for models.
#pipeline: A utility function to create an end-to-end pipeline for various NLP tasks.

In [3]:
model_name = "deepset/roberta-base-squad2" 

#Pre-trained model you are using. deepset/roberta-base-squad2 is a model fine-tuned for the SQuAD2.0 dataset, designed for question-answering tasks

In [4]:
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name, device=0)

#creating a question-answering pipeline using the specified model and tokenizer.

In [5]:
QA_input = {
    'question': 'Why is model conversion important?',
    'context': 'The option to convert models between FARM and transformers gives freedom to the user and let people easily switch between frameworks.'
}

#question: The question you want to ask.
#context: The passage or text from which the answer will be extracted.

In [6]:
res = nlp(QA_input)

#Applies the pipeline to the input dictionary, performing the question-answering task

print(res['answer'])

gives freedom to the user


In [7]:
QA_input2 = {
    'question': 'What are some examples of traditional Indian painting styles?',
    'context': """Indian art and culture have a rich and diverse history that spans thousands of years.
    One of the most prominent features of Indian art is its intricate and symbolic nature, seen in forms 
    ranging from ancient cave paintings in Ajanta and Ellora to the detailed sculptures of Khajuraho.
    Indian classical dance forms like Bharatanatyam, Kathak, and Odissi reflect the deep connection between art, 
    spirituality, and storytelling, often drawing on themes from Hindu mythology. Similarly, Indian music, 
    both classical (like Hindustani and Carnatic) and folk, plays a significant role in cultural expression, 
    often accompanying religious and social ceremonies. The traditional art of Indian painting, 
    including styles like Madhubani, Warli, and Pattachitra, uses vibrant colors and motifs that depict nature, 
    mythology, and daily life. Indian architecture, from the grandeur of the Taj Mahal to the intricate carvings of South Indian temples, 
    showcases the skill and aesthetic sensibilities of Indian artisans. 
    Indian culture is not just a celebration of visual arts but also includes a rich tradition of crafts such as pottery, 
    weaving, and jewelry making, which have been passed down through generations."""
}


In [8]:
res2 = nlp(QA_input2)
print(res2['answer'])

Madhubani, Warli, and Pattachitra


## Lets try with Large chunks of Data

In [9]:
from transformers import pipeline

# Load the pre-trained model and tokenizer
qa_pipeline = pipeline("question-answering", model="deepset/roberta-base-squad2",device=0)

In [10]:
# Paragraph (20 sentences) as context
context = """
Artificial Intelligence (AI) is revolutionizing various industries, ranging from healthcare to finance.
In healthcare, AI systems are being used to predict patient outcomes and personalize treatment plans.
Machine learning algorithms analyze vast amounts of data to find patterns that humans might miss.
For example, AI is helping doctors diagnose diseases like cancer at an early stage.
In finance, AI-powered algorithms are improving trading strategies and risk management.
They analyze market trends and make decisions faster than human traders.
Additionally, AI is being used in customer service, where chatbots provide instant responses to customer queries.
Self-driving cars are another significant advancement, utilizing AI to navigate roads and avoid obstacles.
These cars use a combination of sensors, cameras, and AI algorithms to operate safely.
AI is also playing a role in education, where personalized learning platforms adapt to the needs of individual students.
However, the rise of AI raises ethical concerns, such as the potential for job displacement and privacy issues.
AI systems can inadvertently perpetuate biases present in their training data.
Therefore, it's crucial to ensure that AI models are trained on diverse datasets.
In the field of robotics, AI is enhancing the capabilities of robots, allowing them to perform complex tasks.
AI in robotics is used in manufacturing, where robots work alongside humans.
Meanwhile, in the entertainment industry, AI is being used to create personalized content recommendations.
Social media platforms employ AI algorithms to suggest posts and ads that are likely to engage users.
AI is even being used to compose music and write news articles.
As AI continues to evolve, it is expected to unlock new opportunities and transform the way we live and work.
Despite the numerous benefits, there are calls for regulations to ensure AI is developed responsibly.
Researchers are exploring ways to make AI more transparent and accountable. Ultimately, the goal is to harness AI for the betterment of society.
"""

# Define the question
question = "How is AI being used in the healthcare industry?"

# Split the context into smaller chunks for the model
# Each chunk should ideally be under the model's token limit (approx. 512 tokens)
# Here we split by sentences for simplicity, adjust based on token count if needed
chunks = [
    "Artificial Intelligence (AI) is revolutionizing various industries, ranging from healthcare to finance. In healthcare, AI systems are being used to predict patient outcomes and personalize treatment plans. Machine learning algorithms analyze vast amounts of data to find patterns that humans might miss. For example, AI is helping doctors diagnose diseases like cancer at an early stage.",
    "In finance, AI-powered algorithms are improving trading strategies and risk management. They analyze market trends and make decisions faster than human traders. Additionally, AI is being used in customer service, where chatbots provide instant responses to customer queries. Self-driving cars are another significant advancement, utilizing AI to navigate roads and avoid obstacles.",
    "These cars use a combination of sensors, cameras, and AI algorithms to operate safely. AI is also playing a role in education, where personalized learning platforms adapt to the needs of individual students. However, the rise of AI raises ethical concerns, such as the potential for job displacement and privacy issues. AI systems can inadvertently perpetuate biases present in their training data.",
    "Therefore, it's crucial to ensure that AI models are trained on diverse datasets. In the field of robotics, AI is enhancing the capabilities of robots, allowing them to perform complex tasks. AI in robotics is used in manufacturing, where robots work alongside humans. Meanwhile, in the entertainment industry, AI is being used to create personalized content recommendations.",
    "Social media platforms employ AI algorithms to suggest posts and ads that are likely to engage users. AI is even being used to compose music and write news articles. As AI continues to evolve, it is expected to unlock new opportunities and transform the way we live and work. Despite the numerous benefits, there are calls for regulations to ensure AI is developed responsibly."
]

In [11]:
# Function to get answers from each chunk
def get_answers(question, chunks):
    answers = []
    for chunk in chunks:
        result = qa_pipeline({'question': question, 'context': chunk})
        answers.append((result['answer'], result['score']))
    return answers

# Get answers and confidence scores from each chunk
answers = get_answers(question, chunks)

In [12]:
# Print answers and scores for each chunk
for idx, (answer, score) in enumerate(answers):
    print(f"Chunk {idx+1} Answer: {answer} (Confidence: {score:.2f})")

# Select the answer with the highest confidence score
best_answer = max(answers, key=lambda x: x[1])

print("\nBest Answer:", best_answer[0])
print("Confidence Score:", best_answer[1])

Chunk 1 Answer: to predict patient outcomes and personalize treatment plans (Confidence: 0.67)
Chunk 2 Answer: customer service (Confidence: 0.00)
Chunk 3 Answer: personalized learning platforms adapt to the needs of individual students (Confidence: 0.00)
Chunk 4 Answer: to create personalized content recommendations (Confidence: 0.00)
Chunk 5 Answer: compose music and write news articles (Confidence: 0.00)

Best Answer: to predict patient outcomes and personalize treatment plans
Confidence Score: 0.6696314811706543


## Lets try with a models that can process large number of tokens

### 1. Longformer Model

In [13]:
from transformers import LongformerTokenizer, LongformerForQuestionAnswering, pipeline

# Load Longformer model and tokenizer
tokenizer = LongformerTokenizer.from_pretrained("valhalla/longformer-base-4096-finetuned-squadv1")
model = LongformerForQuestionAnswering.from_pretrained("valhalla/longformer-base-4096-finetuned-squadv1")

# Initialize the QA pipeline with the Longformer model
qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer,device=0)

# Define the question and the context
context = """
Artificial Intelligence (AI) is revolutionizing various industries, ranging from healthcare to finance.
In healthcare, AI systems are being used to predict patient outcomes and personalize treatment plans.
Machine learning algorithms analyze vast amounts of data to find patterns that humans might miss.
For example, AI is helping doctors diagnose diseases like cancer at an early stage.
In finance, AI-powered algorithms are improving trading strategies and risk management.
They analyze market trends and make decisions faster than human traders.
Additionally, AI is being used in customer service, where chatbots provide instant responses to customer queries.
Self-driving cars are another significant advancement, utilizing AI to navigate roads and avoid obstacles.
These cars use a combination of sensors, cameras, and AI algorithms to operate safely.
AI is also playing a role in education, where personalized learning platforms adapt to the needs of individual students.
However, the rise of AI raises ethical concerns, such as the potential for job displacement and privacy issues.
AI systems can inadvertently perpetuate biases present in their training data.
Therefore, it's crucial to ensure that AI models are trained on diverse datasets.
In the field of robotics, AI is enhancing the capabilities of robots, allowing them to perform complex tasks.
AI in robotics is used in manufacturing, where robots work alongside humans.
Meanwhile, in the entertainment industry, AI is being used to create personalized content recommendations.
Social media platforms employ AI algorithms to suggest posts and ads that are likely to engage users.
AI is even being used to compose music and write news articles.
As AI continues to evolve, it is expected to unlock new opportunities and transform the way we live and work.
Despite the numerous benefits, there are calls for regulations to ensure AI is developed responsibly.
Researchers are exploring ways to make AI more transparent and accountable. Ultimately, the goal is to harness AI for the betterment of society.
"""

question = "How is AI being used in the healthcare industry?"

# Get the answer
result = qa_pipeline({'question': question, 'context': context})
print(f"Answer: {result['answer']}")


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/757 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/595M [00:00<?, ?B/s]

Some weights of the model checkpoint at valhalla/longformer-base-4096-finetuned-squadv1 were not used when initializing LongformerForQuestionAnswering: ['longformer.pooler.dense.bias', 'longformer.pooler.dense.weight']
- This IS expected if you are initializing LongformerForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Input ids are automatically padded to be a multiple of `config.attention_window`: 512


Answer: predict patient outcomes and personalize treatment plans.
